In [ ]:
import os
os.environ["NCCL_P2P_DISABLE"] = "2"
os.environ["NCCL_IB_DISABLE"] = "2"
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import torch

# 1. 설정
max_seq_length = 2048
dtype = None
load_in_4bit = True

# 2. 모델 및 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# LoRA 어댑터 추가
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

ModuleNotFoundError: No module named 'unsloth'

In [ ]:
# 3. 데이터셋 준비
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# [중요 변경] 전체 데이터셋 사용
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# small_dataset = dataset.select(range(50))  <-- 이 부분을 제거하여 데이터셋 구조 유지
dataset = dataset.map(formatting_prompts_func, batched = True)

# 4. 트레이너 설정
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # 다시 원래대로 4로 설정해도 괜찮을 것입니다.
        warmup_steps = 5,
        # [중요 변경] 전체 데이터 대신 30 스텝만 빠르게 학습하고 종료하도록 설정
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

# 학습 시작
print("학습을 시작합니다 (max_steps=30)...")
trainer_stats = trainer.train()
print("학습 완료!")